In [2]:
from framework_pkg.framework import FrameWork
from framework_pkg.framework import bin_prediction, ideal_exposure

import numpy as np
from cobaya.run import run

import warnings
warnings.filterwarnings('ignore')


In [4]:
param ={'SinT12'  : 0.310 ,
        'T13'  : 8.57, 
        'mu1'  : 0.,
        'mu2'  : 0., 
        'mu3'  : 0.  ,
        'mdm'  : 1.089e-3 , 
        'alpha': 0.  ,
        'eps' : 0.  ,
        'alpha_eps' : 0.  ,
        'M12'  : 6.9e-5 }


fw_total  = FrameWork(threshold=3.5,bin_max=13, lat=36, efficiency_correction=True, resolution_correction=False)
total_days = fw_total.time_day
experiment_lat = fw_total.lat

exposure = ideal_exposure(experiment_lat)

data_day = 2.284
error_day = np.sqrt(0.020**2 + 0.032**2)

data_night = 2.341
error_night = np.sqrt(0.019**2 + 0.033**2)

In [7]:
def LogLikelihood(sint12, m12, mu2, eps):
    param['SinT12'] = sint12
    param['M12'] = m12 * 1e-5

    param['mu2'] = mu2
    param['alpha'] = 0.73
    param['eps'] = eps
    param['alpha_eps'] = 2.24

    param['mdm'] = 0.84 * 2e-4

    raw_result = fw_total.__getitem__(param)
    R_day, R_night, _ = bin_prediction(raw_result, exposure, total_days)

    return -0.5 * ( (R_day[0,0] - data_day)**2/error_day**2 + (R_night[0,0] - data_night)**2/error_night**2  ) 

In [8]:
print(LogLikelihood(0.308, 6.9, 0., 0.))

print(LogLikelihood(0.310, 7.5, 0.11, 0.74))

-0.03490369741861208
-1.3499309952857197


In [ ]:
info = {"likelihood": {"Chi2": LogLikelihood},
        
        "params": dict([("sint12" , {"prior" : {"dist": 'norm', "loc": 0.310, "scale" : 0.012},
                                  "latex" : r"\sin(\theta_{12})^2"  
                                  }),

                        
                        ("m12"    , {"prior" : {"dist": 'norm', "loc": 7.51, "scale": 0.19},
                                  "latex": r"\Delta m^2_{21} \ \rm 10^{-5} \ eV^2"  
                                  }), 
                        
                        ("mu2"    , {"prior" : {"min": 0, "max": 0.15},
                                   "latex" : r"\tilde{\mu}^2/2 "  
                                   }),
                        
                        # ("alpha"  , {"prior" : {"min": 0, "max": np.pi},
                        #            "latex" : r"\alpha "  
                        #            }),

                      
                        ("eps"  , {"prior" : {"min": 0, "max": 1.},
                                   "latex" : r"\epsilon "  
                                   })

                        # ("alpha_eps"  , {"prior" : {"min": 0, "max": np.pi},
                        #            "latex" : r"\alpha_\epsilon "  
                        #            }), 


                        # ("mdm", {"prior" : {"min": 1, "max": 10},
                        #            "latex": r"m_{\rm dm}"  
                        #           }) 
                        
                        ]),
        
        "sampler": {"mcmc": {"Rminus1_stop": 0.01, "max_tries": 100000 }},
        "output" : "output10/run_info"
       }

In [7]:
#updated_info,sampler = run(info)